In [44]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, sum as _sum, desc, monotonically_increasing_id, lit, month, avg, stddev, when, lit
from pyspark.sql.window import Window
from pyspark.sql.types import DoubleType, DateType

In [15]:
spark = SparkSession.builder \
    .appName("hym") \
    .master("local[2]") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

## Carga y limpieza de datos

In [16]:
ORIGEN_DATOS = "./data"

In [17]:
df_transactions = spark.read.parquet("/home/jovyan/data/transactions.parquet", header=True, inferSchema=True)

In [82]:
productos_df = spark.read.csv("/home/jovyan/data/articles.csv", header=True, inferSchema=True)

In [18]:
df_transactions.printSchema()
df_transactions.show(5)

root
 |-- t_dat: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- article_id: long (nullable = true)
 |-- price: double (nullable = true)
 |-- sales_channel_id: long (nullable = true)

+----------+--------------------+----------+------------------+----------------+
|     t_dat|         customer_id|article_id|             price|sales_channel_id|
+----------+--------------------+----------+------------------+----------------+
|2018-09-20|000058a12d5b43e67...| 663713001|0.0508305084745762|               2|
|2018-09-20|000058a12d5b43e67...| 541518023|0.0304915254237288|               2|
|2018-09-20|00007d2de826758b6...| 505221004|0.0152372881355932|               2|
|2018-09-20|00007d2de826758b6...| 685687003|0.0169322033898305|               2|
|2018-09-20|00007d2de826758b6...| 685687004|0.0169322033898305|               2|
+----------+--------------------+----------+------------------+----------------+
only showing top 5 rows



In [19]:
df_transactions = df_transactions.dropna(subset=["article_id", "price", "t_dat"])

In [7]:
# Convertir price a double y t_dat a fecha
df_transactions = df_transactions.withColumn("price", col("price").cast(DoubleType()))
df_transactions = df_transactions.withColumn("t_dat", to_date("t_dat", "yyyy-MM-dd"))

df_transactions.printSchema()
df_transactions.show(5)

root
 |-- t_dat: date (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- article_id: long (nullable = true)
 |-- price: double (nullable = true)
 |-- sales_channel_id: long (nullable = true)

+----------+--------------------+----------+------------------+----------------+
|     t_dat|         customer_id|article_id|             price|sales_channel_id|
+----------+--------------------+----------+------------------+----------------+
|2018-09-20|000058a12d5b43e67...| 663713001|0.0508305084745762|               2|
|2018-09-20|000058a12d5b43e67...| 541518023|0.0304915254237288|               2|
|2018-09-20|00007d2de826758b6...| 505221004|0.0152372881355932|               2|
|2018-09-20|00007d2de826758b6...| 685687003|0.0169322033898305|               2|
|2018-09-20|00007d2de826758b6...| 685687004|0.0169322033898305|               2|
+----------+--------------------+----------+------------------+----------------+
only showing top 5 rows



In [20]:
df_transactions = df_transactions.dropDuplicates()

In [21]:
df_transactions = df_transactions.filter(col("price") > 0)

## Análisis exploratorio de ventas

Entender los patrones de ventas tanto por artículo como por cliente antes de segmentar.

### Ventas por artículo

En esta sección agrupamos las transacciones por article_id y calculamos la suma de precios (ventas_totales). Esto nos permite identificar cuáles son los artículos que generan mayores ingresos.

In [24]:
ventas_por_articulo = df_transactions.groupBy("article_id") \
    .agg(_sum("price").alias("ventas_totales")) \
    .orderBy(desc("ventas_totales"))

In [25]:
ventas_por_articulo.limit(10).show()

+----------+-----------------+
|article_id|   ventas_totales|
+----------+-----------------+
| 706016001|1382.011830508448|
| 706016002|999.6281694915132|
| 568601006|814.2092711864365|
| 448509014|651.9398644067832|
| 720125001| 615.029305084746|
| 399223001|607.2593389830512|
| 706016003|606.3822203389833|
| 562245046|569.2069152542371|
| 751471001|510.2263050847457|
| 661794001|488.0907118644064|
+----------+-----------------+



### Ventas por cliente

Aquí agrupamos por customer_id para calcular cuánto ha gastado cada cliente en total. Esto ayuda a reconocer a los clientes más valiosos (los que más compran).

In [30]:
ventas_por_cliente = df_transactions.groupBy("customer_id") \
    .agg(_sum("price").alias("gasto_total")) \
    .orderBy(desc("gasto_total"))

In [31]:
ventas_por_cliente.show(10)

+--------------------+------------------+
|         customer_id|       gasto_total|
+--------------------+------------------+
|be1981ab818cf4ef6...| 49.36140677966093|
|a65f77281a528bf5c...| 49.20459322033892|
|b4db5e5259234574e...| 44.20322033898298|
|cd04ec2726dd58a8c...| 40.57903389830502|
|191071b0e1f2e94a5...|40.490186440677924|
|77db96923d20d4053...| 39.78423728813556|
|6cc121e5cc202d2bf...|39.672813559321995|
|f137c16fd17527192...|39.265389830508425|
|03d0011487606c37c...| 37.49713559322029|
|863f0e03da282ae32...|  36.9677796610169|
+--------------------+------------------+
only showing top 10 rows



## ABC Segmentation

Objetivo: Clasificar los artículos en A, B o C según su contribución a las ventas totales.

A: Pocos artículos generan ~70-80% de las ventas.
B: Artículos medianos, 15-25%.
C: El resto (baja contribución).

Clasificamos artículos en A, B o C según su contribución acumulada en ventas.

In [95]:
# Total de ventas
total_ventas = ventas_por_articulo.agg(_sum("ventas_totales")).collect()[0][0]

# Calcular % y % acumulado
window_spec = Window.orderBy(desc("ventas_totales"))

ventas_abc = ventas_por_articulo \
    .withColumn("porcentaje", col("ventas_totales") / total_ventas * 100) \
    .withColumn("porcentaje_acumulado", _sum("porcentaje").over(window_spec)) \
    .withColumn(
        "categoria_ABC",
        when(col("porcentaje_acumulado") <= 80, "A")
        .when(col("porcentaje_acumulado") <= 95, "B")
        .otherwise("C")
    )

# Agregar columna del nombre del producto
ventas_abc = ventas_abc.join(
    productos_df.select("article_id", "prod_name"),
    on="article_id",
    how="left"
)

In [97]:
ventas_abc.select(
    "article_id", "prod_name", "ventas_totales", "porcentaje", "porcentaje_acumulado", "categoria_ABC"
).show(20)

+----------+--------------------+------------------+--------------------+--------------------+-------------+
|article_id|           prod_name|    ventas_totales|          porcentaje|porcentaje_acumulado|categoria_ABC|
+----------+--------------------+------------------+--------------------+--------------------+-------------+
| 706016001|Jade HW Skinny De...| 1382.011830508448| 0.17297676547168372| 0.17297676547168372|            A|
| 706016002|Jade HW Skinny De...| 999.6281694915132| 0.12511647412555563|  0.2980932395972393|            A|
| 568601006|     Mariette Blazer| 814.2092711864365| 0.10190888604410238|  0.4000021256413417|            A|
| 448509014|Perrie Slim Mom D...| 651.9398644067832| 0.08159875808417986| 0.48160088372552157|            A|
| 720125001|   SUPREME RW tights|  615.029305084746| 0.07697892124752433|  0.5585798049730459|            A|
| 399223001|Curvy Jeggings HW...| 607.2593389830512|  0.0760064088750352|  0.6345862138480811|            A|
| 706016003|Jade HW

## XYZ Analysis (variabilidad de la demanda)

Objetivo: Medir la estabilidad de la demanda de cada artículo usando el coeficiente de variación (CV = σ/μ).

X: Demanda estable (baja variación).
Y: Variación moderada.
Z: Muy variable/incierta.

In [37]:
# Crear columna 'month' a partir de 't_dat'
df_transactions = df_transactions.withColumn("month", month("t_dat"))
df_transactions.show(5)

+----------+--------------------+----------+------------------+----------------+-----+
|     t_dat|         customer_id|article_id|             price|sales_channel_id|month|
+----------+--------------------+----------+------------------+----------------+-----+
|2018-09-20|00d781e94d9a533dc...| 568858001|0.1186271186440677|               2|    9|
|2018-09-20|02e45757a6bc483ef...| 537688014|0.0508305084745762|               1|    9|
|2018-09-20|03264406f2f5ad204...| 552617006|0.0287966101694915|               1|    9|
|2018-09-20|06e049f19c81c205a...| 305304008|0.0109661016949152|               2|    9|
|2018-09-20|07bb7488546134d6c...| 671553002|0.0169322033898305|               1|    9|
+----------+--------------------+----------+------------------+----------------+-----+
only showing top 5 rows



In [85]:
# Agrupar ventas por artículo y mes
ventas_mensuales = df_transactions.groupBy("article_id", "month") \
    .agg(_sum("price").alias("ventas_mes"))

# Calcular media y desviación por artículo
xyz_stats = ventas_mensuales.groupBy("article_id") \
    .agg(
        avg("ventas_mes").alias("mean_ventas"),
        stddev("ventas_mes").alias("std_ventas")
    )

# Calcular coeficiente de variación
xyz_stats = xyz_stats.withColumn("cv", col("std_ventas") / col("mean_ventas"))

# Clasificar en X, Y, Z
xyz_stats = xyz_stats.withColumn(
    "XYZ",
    when(col("cv") <= 0.5, lit("X"))
    .when(col("cv") <= 1.0, lit("Y"))
    .otherwise(lit("Z"))
)

# Agregar columna del nombre del producto
xyz_stats = xyz_stats.join(
    productos_df.select("article_id", "prod_name"),
    on="article_id",
    how="left"
)


In [86]:
xyz_stats.select(
    "article_id", "prod_name", "mean_ventas", "std_ventas", "cv", "XYZ"
).show(10)

+----------+--------------------+--------------------+-------------------+------------------+---+
|article_id|           prod_name|         mean_ventas|         std_ventas|                cv|XYZ|
+----------+--------------------+--------------------+-------------------+------------------+---+
| 611020008|Slim Straight 5pk...|  1.3125819209039533| 0.3989496963522421| 0.303942702545752|  X|
| 547300001|      Harley Sweater|  1.0490169491525407| 1.0870725054026042|1.0362773511722638|  Z|
| 673901005|Jade Denim Petite...|   2.478548022598865|  1.760800626052464|0.7104161831838095|  Y|
| 733814003|         Molly dress|   1.075866828087164| 1.4307056756004004|1.3298167005893486|  Z|
| 784485002|              Malina|   2.300063559322029|  1.792946485203219| 0.779520408441109|  Y|
| 739680009|         Mia L/S Tee|0.045599999999999356|0.05307579772843854|1.1639429326412125|  Z|
| 772347001|SKINNY LINED CHECKED| 0.17056308851224086|0.13343979847639728|0.7823486291221835|  Y|
| 788632009|        

## ABC/XYZ Segmentation final

Objetivo: Combinar las dos clasificaciones para tener una matriz de 9 segmentos (AX, AY, AZ, BX, …, CZ).

In [102]:
# Renombrar prod_name en xyz_stats antes del join
xyz_stats_renamed = xyz_stats.withColumnRenamed("prod_name", "prod_name_xyz")

# Join
segmentacion_final = ventas_abc.join(
    xyz_stats_renamed.select("article_id", "XYZ", "prod_name_xyz"), 
    on="article_id", 
    how="inner"
)

# Crear columna Segmento
segmentacion_final = segmentacion_final.withColumn(
    "Segmento",
    concat(col("categoria_ABC"), col("XYZ"))
)

In [101]:
segmentacion_final.select(
    "article_id", "prod_name", "ventas_totales", "categoria_ABC", "XYZ", "Segmento"
).show(20)

+----------+--------------------+-------------------+-------------+---+--------+
|article_id|           prod_name|     ventas_totales|categoria_ABC|XYZ|Segmento|
+----------+--------------------+-------------------+-------------+---+--------+
| 611020008|Slim Straight 5pk...| 15.750983050847438|            A|  X|      AX|
| 547300001|      Harley Sweater| 11.539186440677954|            A|  Z|      AZ|
| 673901005|Jade Denim Petite...| 29.742576271186387|            A|  Y|      AY|
| 733814003|         Molly dress|  7.531067796610147|            B|  Z|      BZ|
| 784485002|              Malina| 27.600762711864334|            A|  Y|      AY|
| 739680009|         Mia L/S Tee|0.22799999999999676|            C|  Z|      CZ|
| 772347001|SKINNY LINED CHECKED|  1.535067796610168|            C|  Y|      CY|
| 788632009|             Douglas| 13.778338983050814|            A|  Z|      AZ|
| 807174001|Lupin padded softbra| 33.386745762711826|            A|  Y|      AY|
| 692002005|       PE CORRES